## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(p=0.3))##Dropout

#輸出
classifier.add(Dense(output_dim=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)


from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop

##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
500/500 [==============================] - 23s 46ms/step - loss: 2.1108 - acc: 0.3641 - val_loss: 1.5723 - val_acc: 0.5259
Epoch 2/100
  9/500 [..............................] - ETA: 7s - loss: 1.7586 - acc: 0.461

C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 20s 39ms/step - loss: 1.6295 - acc: 0.4993 - val_loss: 1.3676 - val_acc: 0.5965
Epoch 3/100
500/500 [==============================] - 20s 40ms/step - loss: 1.4488 - acc: 0.5576 - val_loss: 1.2761 - val_acc: 0.6107
Epoch 4/100
500/500 [==============================] - 20s 40ms/step - loss: 1.3459 - acc: 0.5866 - val_loss: 1.2273 - val_acc: 0.6272
Epoch 5/100
500/500 [==============================] - 20s 40ms/step - loss: 1.2845 - acc: 0.6027 - val_loss: 1.2035 - val_acc: 0.6329
Epoch 6/100
500/500 [==============================] - 20s 41ms/step - loss: 1.2422 - acc: 0.6201 - val_loss: 1.1324 - val_acc: 0.6581
Epoch 7/100
500/500 [==============================] - 20s 40ms/step - loss: 1.2141 - acc: 0.6311 - val_loss: 1.0900 - val_acc: 0.6746
Epoch 8/100
500/500 [==============================] - 20s 40ms/step - loss: 1.1885 - acc: 0.6378 - val_loss: 1.1404 - val_acc: 0.6612
Epoch 9/100
500/500 [==============================] - 20s 40ms/ste

500/500 [==============================] - 20s 40ms/step - loss: 0.9886 - acc: 0.7169 - val_loss: 0.9146 - val_acc: 0.7383
Epoch 63/100
500/500 [==============================] - 20s 40ms/step - loss: 0.9874 - acc: 0.7146 - val_loss: 0.9233 - val_acc: 0.7425
Epoch 64/100
500/500 [==============================] - 20s 39ms/step - loss: 0.9912 - acc: 0.7147 - val_loss: 1.0178 - val_acc: 0.7127
Epoch 65/100
500/500 [==============================] - 20s 39ms/step - loss: 0.9930 - acc: 0.7163 - val_loss: 0.9574 - val_acc: 0.7285
Epoch 66/100
500/500 [==============================] - 20s 39ms/step - loss: 0.9902 - acc: 0.7169 - val_loss: 0.9072 - val_acc: 0.7371
Epoch 67/100
500/500 [==============================] - 20s 39ms/step - loss: 0.9903 - acc: 0.7148 - val_loss: 0.9208 - val_acc: 0.7320
Epoch 68/100
500/500 [==============================] - 20s 39ms/step - loss: 0.9859 - acc: 0.7185 - val_loss: 0.9719 - val_acc: 0.7242
Epoch 69/100
500/500 [==============================] - 20s 3

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.15272042, 0.00214281, 0.5410427 , 0.13824564, 0.03783874,
        0.05103505, 0.05490997, 0.00995078, 0.01122251, 0.00089141]],
      dtype=float32)